In [1]:
import numpy as np
import scipy.spatial as spa
import scipy 
import prody
from pathlib import Path
import importlib
from typing import Union, List, Dict

import rdkit

import dgl
import torch

%matplotlib widget
from matplotlib import pyplot as plt

# define hard-coded filepaths

In [2]:
DEV_INDEX_PATH = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind_processed/train_index_mini.txt'
DEV_DATA_DIR = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind/refined-set'
DEV_PROCESSED_DATA_DIR = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind_processed/processed_data'

DEV_DATA_DIR = Path(DEV_DATA_DIR)

# import code for processing pdbbind

In [3]:
try:
    importlib.reload(process_pdbbind)
    importlib.reload(pdbbind_dataset)
except NameError:
    import data_processing.pdbbind_processing as process_pdbbind
    import data_processing.pdbbind_dataset as pdbbind_dataset

parse_protein = process_pdbbind.parse_protein
parse_ligand = process_pdbbind.parse_ligand
get_pocket_atoms = process_pdbbind.get_pocket_atoms
build_receptor_graph = process_pdbbind.build_receptor_graph

PDBbind = pdbbind_dataset.PDBbind
rec_atom_elements = pdbbind_dataset.rec_atom_elements
lig_atom_elements = pdbbind_dataset.lig_atom_elements

# create dataset object

In [9]:
dataset = PDBbind(name='dev_dataset', 
                index_fpath=DEV_INDEX_PATH, 
                raw_data_dir=DEV_DATA_DIR, 
                processed_data_dir=DEV_PROCESSED_DATA_DIR, 
                rec_elements=rec_atom_elements,
                lig_elements=lig_atom_elements,
                lig_box_padding=6, pocket_cutoff=4,
                receptor_k=3,
                dataset_size=None)

@> 4858 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 10035 atoms and 1 coordinate set(s) were parsed in 0.05s.
@> 9039 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 8961 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> 21443 atoms and 1 coordinate set(s) were parsed in 0.11s.
@> 17244 atoms and 1 coordinate set(s) were parsed in 0.09s.
@> 66431 atoms and 1 coordinate set(s) were parsed in 0.33s.
@> 66974 atoms and 1 coordinate set(s) were parsed in 0.35s.
@> 67572 atoms and 1 coordinate set(s) were parsed in 0.32s.
@> 66997 atoms and 1 coordinate set(s) were parsed in 0.33s.
@> 66629 atoms and 1 coordinate set(s) were parsed in 0.40s.
@> 67081 atoms and 1 coordinate set(s) were parsed in 0.34s.
@> 67306 atoms and 1 coordinate set(s) were parsed in 0.33s.
@> 65693 atoms and 1 coordinate set(s) were parsed in 0.31s.
@> 67850 atoms and 1 coordinate set(s) were parsed in 0.35s.
@> 66346 atoms and 1 coordinate set(s) were parsed in 0.33s.
@> 67542 atoms and 1 coordi